<a href="https://colab.research.google.com/github/Suiii71/ML-Projects/blob/main/Car_Project_3rd_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install torchvision timm

Import Libraries

In [6]:
import os
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

Dataset Class

In [7]:
class CarDamageDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row["image"])
        image = Image.open(img_path).convert("RGB")

        label = row["label"]

        if self.transform:
            image = self.transform(image)

        return image, label

In [8]:
ROOT = "/content/drive/MyDrive/Car_Damage_Project"
TRAIN_CSV = os.path.join(ROOT, "CarDD_classification_train.csv")
TEST_CSV  = os.path.join(ROOT, "CarDD_classification_test.csv")

TRAIN_IMG_DIR = os.path.join(ROOT, "train2017")
TEST_IMG_DIR  = os.path.join(ROOT, "test2017")

df_train = pd.read_csv(TRAIN_CSV)
df_test  = pd.read_csv(TEST_CSV)

le = LabelEncoder()
df_train["label"] = le.fit_transform(df_train["label"])
df_test["label"]  = le.transform(df_test["label"])  # safe
num_classes = len(le.classes_)

train_df, val_df = train_test_split(
    df_train, test_size=0.2, stratify=df_train["label"], random_state=42
)

print("Train:", len(train_df))
print("Val:", len(val_df))
print("Test:", len(df_test))
print("Classes:", le.classes_)

Train: 2252
Val: 564
Test: 374
Classes: ['crack' 'dent' 'glass shatter' 'lamp broken' 'scratch' 'tire flat']


Augmentation

In [9]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(380, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(25),
    transforms.ColorJitter(0.2, 0.2, 0.2),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

Dataloaders

In [10]:
train_dataset = CarDamageDataset(train_df, TRAIN_IMG_DIR, train_transform)
val_dataset   = CarDamageDataset(val_df, TRAIN_IMG_DIR, val_transform)
test_dataset  = CarDamageDataset(df_test, TEST_IMG_DIR, val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32)
test_loader  = DataLoader(test_dataset, batch_size=32)

Computing class weights

In [11]:
class_counts = train_df["label"].value_counts().sort_index().values
weights = torch.tensor(1.0 / class_counts, dtype=torch.float)
weights = weights / weights.sum() * len(class_counts)
weights = weights.to("cuda")

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

Training Function (with Early Stopping + Saving Best Model)

In [ ]:
def train_model(model, train_loader, val_loader, epochs=40):

    criterion = nn.CrossEntropyLoss(weight=weights, label_smoothing=0.1)
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)

    best_acc = 0
    patience = 7
    es_counter = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for imgs, labels in train_loader:
            imgs, labels = imgs.cuda(), labels.cuda()

            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Validation
        model.eval()
        correct, total = 0, 0

        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.cuda(), labels.cuda()
                outputs = model(imgs)
                _, preds = torch.max(outputs, 1)

                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_acc = correct / total
        scheduler.step(1 - val_acc)

        print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss:.4f} | Val Acc: {val_acc:.4f}")

        # early stopping
        if val_acc > best_acc:
            best_acc = val_acc
            es_counter = 0
            torch.save(model.state_dict(), "best_model.pth")
        else:
            es_counter += 1

        if es_counter >= patience:
            print("Early stopping!")
            break

    print("Best Validation Accuracy =", best_acc)
    return best_acc

ResNet50

In [ ]:
resnet = timm.create_model("resnet50", pretrained=True, num_classes=num_classes).cuda()
acc_resnet = train_model(resnet, train_loader, val_loader)

EfficientNetB0

In [ ]:
eff = timm.create_model("efficientnet_b0", pretrained=True, num_classes=num_classes).cuda()
acc_eff = train_model(eff, train_loader, val_loader)

MobileNetV2

In [ ]:
mob = timm.create_model("mobilenetv2_100", pretrained=True, num_classes=num_classes).cuda()
acc_mob = train_model(mob, train_loader, val_loader)

Test Accuracy Evaluation

In [ ]:
def test_accuracy(model_path):
    model = timm.create_model("efficientnet_b0", pretrained=False, num_classes=num_classes).cuda()
    model.load_state_dict(torch.load(model_path))
    model.eval()

    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.cuda(), labels.cuda()
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    print("Test Accuracy =", correct / total)

test_accuracy("best_model.pth")